In [207]:
import mysql.connector as mysql
from config import host, user, password
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import calendar, math
pd.options.mode.chained_assignment = None

In [9]:
con = mysql.connect(user=user,password=password,host=host,database='team3')
cursor = con.cursor()

## Spis treści
### [1) Najlepsi gracze w naszym klubie](#1-bullet)
### [2) Mistrzostwa w curling'u](#2-bullet)
### [3) ](#3-bullet)
### [4) ](#4-bullet)
### [5) ](#5-bullet)
### [6) ](#6-bullet)
### [7) ](#7-bullet)

<a name="1-bullet"></a>
## Najlepsi gracze w naszym klubie

Na podstawie graczy którzy brali udział w meczu na jakiejś pozycji (jeśli mają przypisaną pozycję, to grali a nie siedzieli), szukamy tych którzy mają największą skuteczność (ilość wygranych do wszystkich meczy). Szukamy wśród dorosłych i juniorów oddzielnie, zaczynając od dorosłych.

### Najlepsi dorośli gracze

In [151]:
def display_best_players(data, number):
    data['success_rate'] = data['won']/data['played']
    data['success_rate'] = round(data['success_rate'].astype(float), 5)
    data = data[['first_name','last_name','won', 'played', 'success_rate']]
    data = data.sort_values(by=['success_rate', 'played'], ascending=False)
    data.reset_index(drop=True, inplace=True)
    data.index = data.index + 1
    return data[:number]

In [167]:
def display_player_info(data):
    today = pd.to_datetime('2022-05-30')
    for index, row in data.iterrows():
        age = math.floor((today -  pd.to_datetime(row['birthdate']))/np.timedelta64(1, 'Y'))
        if row['gender'] == 'M':
            gender = 'mężczyzna'
        else:
            gender = 'kobieta'
        print('{}) {} {}, {} w wieku {} lat, z drużyny {}'.format(index+1, row['first_name'], row['last_name'], gender, age, row['team_name']))

In [140]:
query_best_adults = '''
SELECT SUM(team_score > opponent_score) AS won, COUNT(team_score > opponent_score) AS played, first_name, last_name 
FROM positions
LEFT JOIN people USING (person_id) 
LEFT JOIN personal_info USING (person_id) 
LEFT JOIN matches USING (team_name, date) 
LEFT JOIN teams USING (team_name)
WHERE team_score IS NOT NULL AND age_category = 'adult' 
GROUP BY person_id
'''

In [141]:
cursor.execute(query_best_adults)
result_adults = cursor.fetchall()
data_adults = pd.DataFrame(result_adults,columns=['won', 'played','first_name','last_name'])

In [154]:
display_best_players(data_adults, 40)

,first_name,last_name,won,played,success_rate
1,Zacharia,Donaldson,15,15,1.00000
2,Abdurrahman,Huynh,13,13,1.00000
3,Alexandro,Guerin,11,11,1.00000
4,Dorian,Lam,11,11,1.00000
5,Nick,Small,10,10,1.00000
6,Manley,Gravel,9,9,1.00000
7,Eduardo,Costa,7,7,1.00000
8,Gena,Vezina,5,5,1.00000
9,Glen,Christie,5,5,1.00000
10,Reno,Duchesne,4,4,1.00000


In [157]:
query_adult_info = '''
SELECT first_name, last_name, gender, birthdate, team_name
FROM people
LEFT JOIN personal_info USING (person_id) 
LEFT JOIN gender USING (first_name) 
WHERE (first_name = 'Zacharia' AND last_name = 'Donaldson') OR 
    (first_name = 'Abdurrahman' AND last_name = 'Huynh') OR 
    (first_name = 'Alexandro' AND last_name = 'Guerin') OR 
    (first_name = 'Dorian' AND last_name = 'Lam') OR 
    (first_name = 'Nick' AND last_name = 'Small')
GROUP BY person_id
'''

In [158]:
cursor.execute(query_adult_info)
info_adults = cursor.fetchall()
info_adults = pd.DataFrame(info_adults,columns=['first_name', 'last_name', 'gender', 'birthdate', 'team_name'])

Dość spodziewane było pojawienie się w czołówce graczy którzy grali niewiele meczy i udało im się je wygrać, dlatego wyświetlona jest również liczba meczy oraz sortowanie po niej by móc stwierdzić, dla których zwycięstwa nie są przypadkiem.
W powyższej tabeli widzimy kilku świetnych graczy z $100\%$ skutecznością, a grali oni w ponad jednym meczu! <b>Najlepsza 5 z dorosłych graczy w CurlingMasters to:</b>	

In [168]:
display_player_info(info_adults)

1) Alexandro Guerin, mężczyzna w wieku 29 lat, z drużyny Calgary Rockets
2) Zacharia Donaldson, mężczyzna w wieku 42 lat, z drużyny Calgary Rockets
3) Abdurrahman Huynh, mężczyzna w wieku 45 lat, z drużyny Calgary Rockets
4) Nick Small, mężczyzna w wieku 49 lat, z drużyny Calgary Rockets
5) Dorian Lam, mężczyzna w wieku 36 lat, z drużyny Calgary Rockets


Okazuje się, że panowie pochodzą z jednej drużyny, jest to zatem mistrzowska drużyna! Warto również zwrócić uwagę na takich zawodników jak: Akhil Berube, Sathana Lo, Emile Burton, Juliano Lord, Ankit Cousineau, których wyniki są również obiecujące, ze względu na ilość rozegranych przez nich meczy.

### Najlepsi juniorzy

In [143]:
query_best_juniors = '''
SELECT SUM(team_score > opponent_score) AS won, COUNT(team_score > opponent_score) AS played, first_name, last_name 
FROM positions
LEFT JOIN people USING (person_id) 
LEFT JOIN personal_info USING (person_id) 
LEFT JOIN matches USING (team_name, date) 
LEFT JOIN teams USING (team_name) 
WHERE team_score IS NOT NULL AND age_category = 'junior' 
GROUP BY person_id
'''

In [144]:
cursor.execute(query_best_juniors)
result_junior = cursor.fetchall()
data_juniors = pd.DataFrame(result_junior,columns=['won', 'played','first_name','last_name'])

In [169]:
display_best_players(data_juniors, 40)

,first_name,last_name,won,played,success_rate
1,Israel,Chevalier,13,13,1.0
2,Daylen,Larsen,13,13,1.0
3,Craig,McCarthy,13,13,1.0
4,Carl,Perron,13,13,1.0
5,Jahaan,Turgeon,9,9,1.0
6,Mackayla,Little,8,8,1.0
7,Petra,Reimer,8,8,1.0
8,Rania,Faucher,6,6,1.0
9,Fisher,Aubin,6,6,1.0
10,Ioanna,Brar,6,6,1.0


In [170]:
query_junior_info = '''
SELECT first_name, last_name, gender, birthdate, team_name
FROM people
LEFT JOIN personal_info USING (person_id) 
LEFT JOIN gender USING (first_name) 
WHERE (first_name = 'Israel' AND last_name = 'Chevalier') OR 
    (first_name = 'Daylen' AND last_name = 'Larsen') OR 
    (first_name = 'Craig' AND last_name = 'McCarthy') OR 
    (first_name = 'Carl' AND last_name = 'Perron') OR 
    (first_name = 'Jahaan' AND last_name = 'Turgeon')
GROUP BY person_id
'''

In [173]:
cursor.execute(query_junior_info)
info_juniors = cursor.fetchall()
info_juniors = pd.DataFrame(info_juniors,columns=['first_name', 'last_name', 'gender', 'birthdate', 'team_name'])

Juniorów z idealną skutecznością mamy jeszcze więcej niż dorosłych. <b>Przyrzyjmy się dokładniej top 5:</b>

In [174]:
display_player_info(info_juniors)

1) Israel Chevalier, mężczyzna w wieku 20 lat, z drużyny Montreal Braves
2) Daylen Larsen, mężczyzna w wieku 17 lat, z drużyny Montreal Braves
3) Craig McCarthy, mężczyzna w wieku 10 lat, z drużyny Montreal Braves
4) Carl Perron, mężczyzna w wieku 10 lat, z drużyny Montreal Braves
5) Jahaan Turgeon, mężczyzna w wieku 9 lat, z drużyny Toronto Olympics


Widzimy, że najlepsi zawodnicy, są to mężczyźni, znów w większości z jednej drużyny. Jest to najprawdopdobniej spowodowane przez przypisywanie skuteczności (wykorzystywanej do generacji wyników meczu) do drużyny. Dlatego jeśli drużyna ma wysoką skuteczność, oraz rozegrała sporo meczy, to właśnie jej członkowie pojawią się wysoko i obok siebie w rankingu. Warto zwrócic uwagę na wielki potencjał Jahaan Turgeon, który już w wieku $9$ lat pojawił się w top $5$.

<a name="2-bullet"></a>
## Mistrzostwa w curling'u

Chcemy wybrać najlepszych zawodników do mistrzowskich drużyn, które będzie reprezentować cały nasz klub. Nie zwracamy uwagi na miasta, czy drużyny graczy bo chcemy dobrać najlepszą możliwą osobę na dane stanowisko w drużynie. Graczy będziemy wybierać do drużyny męskiej lub żeńskiej w zależności od kategori wiekowej.

In [214]:
def best_for_position(data, position):
    data = data[data['position'] == position]
    data['success_rate'] = data['won']/data['played']
    data['success_rate'] = round(data['success_rate'].astype(float), 5)
    today = pd.to_datetime('2022-05-30')
    age = []
    for index, row in data.iterrows():
        age.append(math.floor((today -  pd.to_datetime(row['birthdate']))/np.timedelta64(1, 'Y')))
    data['age'] = age
    data = data[['first_name', 'last_name', 'position','played', 'success_rate', 'gender', 'age', 'team_name']]
    data = data.sort_values(by=['success_rate', 'played'], ascending=False)
    data.reset_index(drop=True, inplace=True)
    data.index = data.index + 1
    return data[:5]

### Najlepsi dorośli mężczyźni w zależności od pozycji na której grali

In [237]:
query_adult_m = '''
SELECT first_name, last_name, positions.position, SUM(team_score > opponent_score), COUNT(team_score > opponent_score), gender, birthdate, team_name 
FROM positions
LEFT JOIN people USING (person_id) 
LEFT JOIN gender USING (first_name) 
LEFT JOIN personal_info USING (person_id) 
LEFT JOIN matches USING (team_name, date) 
LEFT JOIN teams USING (team_name) 
WHERE (team_score IS NOT NULL) AND (age_category = 'adult') AND (gender = 'M')
GROUP BY person_id, positions.position
'''

In [238]:
cursor.execute(query_adult_m)
team_adult_m = cursor.fetchall()
team_adult_m = pd.DataFrame(team_adult_m, columns=['first_name', 'last_name', 'position', 'won', 'played', 'gender', 'birthdate', 'team_name'])

In [239]:
best_for_position(team_adult_m, 'lead')

,first_name,last_name,position,played,success_rate,gender,age,team_name
1,Emile,Burton,lead,9,1.0,M,42,Montreal Bruins
2,Zacharia,Donaldson,lead,7,1.0,M,42,Calgary Rockets
3,Ankit,Cousineau,lead,7,1.0,M,37,Montreal Bruins
4,Alexandro,Guerin,lead,5,1.0,M,29,Calgary Rockets
5,Juliano,Lord,lead,5,1.0,M,32,Vancouver Fury


In [240]:
best_for_position(team_adult_m, 'vice')

,first_name,last_name,position,played,success_rate,gender,age,team_name
1,Stavros,Thiessen,vice,6,1.0,M,38,Cold Colts
2,Marcelo,Dion,vice,6,1.0,M,25,Vancouver Fury
3,Gordon,Klassen,vice,6,1.0,M,27,Montreal Bruins
4,Johan,Webb,vice,5,1.0,M,28,Calgary Bruins
5,Nick,Small,vice,5,1.0,M,49,Calgary Rockets


In [241]:
best_for_position(team_adult_m, 'second')

,first_name,last_name,position,played,success_rate,gender,age,team_name
1,Thierry,Donnelly,second,11,1.0,M,39,Montreal Bruins
2,Daryn,Labelle,second,7,1.0,M,37,Vancouver Fury
3,Yanick,Dhaliwal,second,6,1.0,M,37,Montreal Bruins
4,Abdurrahman,Huynh,second,4,1.0,M,45,Calgary Rockets
5,Jake,Morris,second,4,1.0,M,46,Cold Huskies


In [242]:
best_for_position(team_adult_m,'skip') 

,first_name,last_name,position,played,success_rate,gender,age,team_name
1,Gordon,Klassen,skip,7,1.0,M,27,Montreal Bruins
2,Edsel,Green,skip,5,1.0,M,47,Cold Colts
3,Dhyan,Fillion,skip,5,1.0,M,27,Vancouver Fury
4,Zacharia,Donaldson,skip,4,1.0,M,42,Calgary Rockets
5,Manley,Gravel,skip,4,1.0,M,40,Calgary Rockets


Widzimy, że wśród drużyn dorosłych mężczyzn najlepsi zawodnicy są z drużyn: Montreal Bruins, Calgary Rockets, Vancouver Fury oraz Cold Colts. Niektórzy zawodnicy osiągają bardzo dobre wyniki na więcej niż jednej pozycji np. Gordon Klassen czy Zacharia Donaldson. 

W przypadku tej kategori, najlepszy skład moglibyśmy wybrać z następujących osób:
* lead - Emile Burton,
* vice - Stavros Thiessen lub Marcelo Dion lub Gordon Klassen,
* second - Thierry Donnelly,
* skip - Gordon Klassen.

Widzimy, że wszyscy zawodnicy poza 'vice' są jednoznacznie najlepsi oraz pochodzą z drużyny Montreal Bruins. Na pozycję 'vice', nie możemy wybrać Gordon Klassen, ponieważ jest on już na pozycji 'skip', więc do wyboru mamy dowolnego z pozostałych dwóch graczy. Starvros Thiessen pojawia się jako $38$ najlepszy gracz bez względu na pozycję, przy czym Marcelo Dion nie widać w top $40$, zatem decydujemy się na Starvros Thiessen.

Ostatecznie jako <b>reprezentację dorosłych mężczyzn wybieramy: </b>
* <b>lead - Emile Burton</b> z Montreal Bruins, 
* <b>vice - Stavros Thiessen</b> z Cold Colts,
* <b>second - Thierry Donnelly</b> z Montreal Bruins,
* <b>skip - Gordon Klassen</b> z Montreal Bruins.

### Najlepsze dorosłe kobiety w zależności od pozycji na której grały

In [231]:
query_adult_w = '''
SELECT first_name, last_name, positions.position, SUM(team_score > opponent_score), COUNT(team_score > opponent_score), gender, birthdate, team_name 
FROM positions
LEFT JOIN people USING (person_id) 
LEFT JOIN gender USING (first_name) 
LEFT JOIN personal_info USING (person_id) 
LEFT JOIN matches USING (team_name, date) 
LEFT JOIN teams USING (team_name) 
WHERE (team_score IS NOT NULL) AND (age_category = 'adult') AND (gender = 'F')
GROUP BY person_id, positions.position
'''

In [232]:
cursor.execute(query_adult_w)
team_adult_w = cursor.fetchall()
team_adult_w = pd.DataFrame(team_adult_w, columns=['first_name', 'last_name', 'position', 'won', 'played', 'gender', 'birthdate', 'team_name'])

In [233]:
best_for_position(team_adult_w, 'lead')

,first_name,last_name,position,played,success_rate,gender,age,team_name
1,Najma,Charlebois,lead,7,1.0,F,34,Game of Stones
2,Adãle,Meyer,lead,5,1.0,F,35,Game of Stones
3,Sathana,Lo,lead,3,1.0,F,27,Cold Huskies
4,Anahita,Roy,lead,3,1.0,F,36,Toronto Penguins
5,Humaira,Sharpe,lead,1,1.0,F,40,Ice Cougars


In [234]:
best_for_position(team_adult_w, 'vice')

,first_name,last_name,position,played,success_rate,gender,age,team_name
1,Sara,Willis,vice,6,1.0,F,29,Game of Stones
2,Kristie,Bartlett,vice,4,1.0,F,49,Calgary Bruins
3,Patricia-anne,Audet,vice,3,1.0,F,22,Game of Stones
4,Laiba,Baird,vice,2,1.0,F,48,Cold Huskies
5,Gena,Vezina,vice,1,1.0,F,31,Cold Huskies


In [235]:
best_for_position(team_adult_w, 'second')

,first_name,last_name,position,played,success_rate,gender,age,team_name
1,Preet,Doiron,second,3,1.0,F,39,Game of Stones
2,Laiba,Baird,second,2,1.0,F,48,Cold Huskies
3,Gena,Vezina,second,2,1.0,F,31,Cold Huskies
4,Domenica,Shah,second,2,1.0,F,47,Calgary Bruins
5,Aliah,Hawkins,second,1,1.0,F,26,Ice Cougars


In [236]:
best_for_position(team_adult_w, 'skip')

,first_name,last_name,position,played,success_rate,gender,age,team_name
1,Adãle,Meyer,skip,5,1.0,F,35,Game of Stones
2,Anayat,Major,skip,5,1.0,F,26,Toronto Penguins
3,Sathana,Lo,skip,3,1.0,F,27,Cold Huskies
4,Kelsie,Gallant,skip,2,1.0,F,35,Toronto Penguins
5,Patricia-anne,Audet,skip,2,1.0,F,22,Game of Stones


Dla dorosłych kobiet najlepsze zawodniczki podobnie jak u mężczyzn, pochodzą z powtarzających się drużyn, takich jak: Game of Stones, Cold Huskies, Toronto Penguins, Ice Cougars i Calgary Bruins. Również widzimy zawodniczki świetne bez wzgędu na jakiej pozycji grały np. Adãle Meyer i Sathana Lo.

Bazując na powyższych, najlepszy skład moglibyśmy wybrać z następujących osób:
* lead - Najma Charlebois,
* vice - Sara Willis,
* second - Preet Doiron,
* skip - Adãle Meyer lub Anayat Major.

Wybrane zawodniczki poza 'skip' są z drużyny Game of Stones. Sugerując się drużyną, warto byłoby wybrać Adãle Meyer na tę pozycję, ponieważ najprawdopodobniej zawodniczki które są z jednego teamu, będą w stanie najlepiej razem współpracować.

Zatem jako <b>reprezentację dorosłych kobiet wybieramy drużynę Game of Stones w składzie: </b>
* <b>lead - Najma Charlebois,</b> 
* <b>vice - Sara Willis,</b>
* <b>second - Preet Doiron,</b>
* <b>skip - Adãle Meyer.</b>

### Najlepsi mężczyźni wśród juniorów w zależności od pozycji

In [243]:
query_junior_m = '''
SELECT first_name, last_name, positions.position, SUM(team_score > opponent_score), COUNT(team_score > opponent_score), gender, birthdate, team_name 
FROM positions
LEFT JOIN people USING (person_id) 
LEFT JOIN gender USING (first_name) 
LEFT JOIN personal_info USING (person_id) 
LEFT JOIN matches USING (team_name, date) 
LEFT JOIN teams USING (team_name) 
WHERE (team_score IS NOT NULL) AND (age_category = 'junior') AND (gender = 'M')
GROUP BY person_id, positions.position
'''

In [244]:
cursor.execute(query_junior_m)
team_junior_m = cursor.fetchall()
team_junior_m = pd.DataFrame(team_junior_m, columns=['first_name', 'last_name', 'position', 'won', 'played', 'gender', 'birthdate', 'team_name'])

In [249]:
best_for_position(team_junior_m, 'lead')

,first_name,last_name,position,played,success_rate,gender,age,team_name
1,Israel,Chevalier,lead,5,1.0,M,20,Montreal Braves
2,Mervin,Webb,lead,5,1.0,M,11,Stone Timberwolves
3,Issa,Sharma,lead,4,1.0,M,15,Stone Timberwolves
4,Daylen,Larsen,lead,4,1.0,M,17,Montreal Braves
5,Craig,McCarthy,lead,3,1.0,M,10,Montreal Braves


In [250]:
best_for_position(team_junior_m, 'vice')

,first_name,last_name,position,played,success_rate,gender,age,team_name
1,Daylen,Larsen,vice,7,1.0,M,17,Montreal Braves
2,Olivier,Donovan,vice,6,1.0,M,17,Stone Timberwolves
3,Frederik,Adam,vice,4,1.0,M,11,Super Salsa
4,Mervin,Webb,vice,4,1.0,M,11,Stone Timberwolves
5,Timur,Labelle,vice,3,1.0,M,12,Broom Hawks


In [251]:
best_for_position(team_junior_m, 'second')

,first_name,last_name,position,played,success_rate,gender,age,team_name
1,Tal,Ho,second,7,1.0,M,13,Stone Timberwolves
2,Craig,McCarthy,second,5,1.0,M,10,Montreal Braves
3,Boy,Edwards,second,4,1.0,M,13,Broom Hawks
4,Carl,Perron,second,4,1.0,M,10,Montreal Braves
5,Khaled,David,second,4,1.0,M,8,Super Salsa


In [252]:
best_for_position(team_junior_m, 'skip')

,first_name,last_name,position,played,success_rate,gender,age,team_name
1,Tal,Ho,skip,6,1.0,M,13,Stone Timberwolves
2,Carl,Perron,skip,5,1.0,M,10,Montreal Braves
3,Jahaan,Turgeon,skip,5,1.0,M,9,Toronto Olympics
4,Israel,Chevalier,skip,4,1.0,M,20,Montreal Braves
5,Craig,McCarthy,skip,4,1.0,M,10,Montreal Braves


Ponownie najlepsi zawodnicy są z tylko kilku drużyn, tutaj: Montreal Braves, Stone Timberwolves, Super Salsa, Broom Hawks i Toronto Olympics. Co ciekawe mamy zawodnika (Tal Ho) który jest jednoznacznie najlepszy w dwóch kategoriach. 

Z powyższych odczytujemy nasze możliwe opcje:
* lead - Israel Chevalier lub Mervin Webb,
* vice - Daylen Larsen,	
* second - Tal Ho > Craig McCarthy,
* skip - Tal Ho > Carl Perron.

Najpierw rozwiązujemy kwestię na jaką pozycję trafi Tal Ho, lepsza sytuacja będzie gdzy zajmie on pozycję 'second' a wtedy zamiast niego Carl Perron zostanie 'skip'. Jest tak ponieważ widzimy mniejszą różnicę między umiejętnościami Tal Ho i Carl Perron niż między Tal Ho i Craig McCarthy. Jako 'lead' wybieramy bezsprzecznie Israel Chevalier, który jest naszym najlepszym graczem wśród juniorów!

<b>Reprezentacja męska juniorów prezentuje się następjąco:</b>
* <b>lead - Israel Chevalier</b> z drużyny Montreal Braves,
* <b>vice - Daylen Larsen</b> z drużyny Montreal Braves,
* <b>second - Tal Ho</b> z drużyny Stone Timberwolves,
* <b>skip - Carl Perron</b>z drużyny Montreal Braves.

### Najlepsze kobiety wśród juniorów w zależności od pozycji

In [253]:
query_junior_w = '''
SELECT first_name, last_name, positions.position, SUM(team_score > opponent_score), COUNT(team_score > opponent_score), gender, birthdate, team_name 
FROM positions
LEFT JOIN people USING (person_id) 
LEFT JOIN gender USING (first_name) 
LEFT JOIN personal_info USING (person_id) 
LEFT JOIN matches USING (team_name, date) 
LEFT JOIN teams USING (team_name) 
WHERE (team_score IS NOT NULL) AND (age_category = 'junior') AND (gender = 'F')
GROUP BY person_id, positions.position
'''

In [254]:
cursor.execute(query_junior_w)
team_junior_w = cursor.fetchall()
team_junior_w = pd.DataFrame(team_junior_w, columns=['first_name', 'last_name', 'position', 'won', 'played', 'gender', 'birthdate', 'team_name'])

In [255]:
best_for_position(team_junior_w, 'lead')

,first_name,last_name,position,played,success_rate,gender,age,team_name
1,Elysse,Prevost,lead,5,1.0,F,15,Cold Islanders
2,Adaline,Parsons,lead,4,1.0,F,11,Cold Islanders
3,Female,Hewitt,lead,3,1.0,F,14,Ottawa Greys
4,Chastity,Tessier,lead,3,1.0,F,9,Toronto Olympics
5,Rania,Faucher,lead,2,1.0,F,17,Calgary Metros


In [256]:
best_for_position(team_junior_w, 'vice')

,first_name,last_name,position,played,success_rate,gender,age,team_name
1,Liliane,Freeman,vice,4,1.0,F,11,Montreal Lasers
2,Elysse,Prevost,vice,4,1.0,F,15,Cold Islanders
3,Hester,Newton,vice,4,1.0,F,18,Montreal Lasers
4,Mackayla,Little,vice,3,1.0,F,12,Toronto Olympics
5,Petra,Reimer,vice,3,1.0,F,12,Cold Islanders


In [257]:
best_for_position(team_junior_w, 'second')

,first_name,last_name,position,played,success_rate,gender,age,team_name
1,Ksenia,Godin,second,6,1.0,F,10,Cold Islanders
2,Braelynn,Beaton,second,3,1.0,F,19,Stone Timberwolves
3,Simra,Nielsen,second,3,1.0,F,11,Ice Dodgers
4,Niki,Croteau,second,3,1.0,F,12,Cold Islanders
5,Mykayla,Champagne,second,3,1.0,F,20,Super Atlantics


In [258]:
best_for_position(team_junior_w, 'skip')

,first_name,last_name,position,played,success_rate,gender,age,team_name
1,Adaline,Parsons,skip,5,1.0,F,11,Cold Islanders
2,Leita,Lam,skip,3,1.0,F,13,Montreal Lasers
3,Delma,Davis,skip,3,1.0,F,16,Montreal Lasers
4,Rachel,ONeill,skip,2,1.0,F,10,Montreal Lasers
5,Rania,Faucher,skip,2,1.0,F,17,Calgary Metros


Na koniec szukamy jeszcze reprezentacji kobiet dla juniorów. Jak zwykle mamy pule kilku najlepszych drużyn z których pochodzą nasze zawodniczki, są to: Cold Islanders, Ottawa Greys, Toronto Olympics, Calgary Metros, Montreal Lasers, Stone Timberwolves, Ice Dodger oraz Super Atlantics. Pula tutaj znacznie się nam powiększyła, co znaczy, że drużyny żeńskie są bardziej zróżnicowane. 

W tej kategorii mamy następujące kandydatki:
* lead - Elysse	Prevost,
* vice - Liliane Freeman lub Elysse Prevost albo Hester Newton,	
* second - Ksenia Godin,
* skip - Adaline Parsons.

Jedyna sporna pozycja, to 'vice' z której odpada nam Elysse Prevost, będąca naszym 'lead'. Liliane Freeman i Hester Newton nie pojawiają się w top $40$ zawodników ani żadna nie jest z drużyny pozostałych zawodniczek, więc zupełnie zmieniamy taktykę i wybieramy Petra Reimer. Znajduje się ona w Cold Islanders, jest $7$ najlepszą zawodniczką wśród juniorów, a różnica w jej umiejętnościach i wytypowanych dziewczyn jest niewielka.

Nasza <b>reprezentacja żeńska juniorów pochodzi cała z drużyny Cold Islanders i składa się z:</b>
* <b>lead - Elysse Prevost,</b>
* <b>vice - Petra Reimer,</b>
* <b>second - Ksenia Godin,</b>
* <b>skip - Adaline Parsons.</b>